# Merging Practice

In [8]:
import pandas as pd
# this "firm" variable is really "last firm" but the database manager called it what they called it 
# and now IT is scared that fixing it will break all their systems, so it's "stuck as-is"
some_employees = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Austin'],
                    'firm': ['Citi','GS','Accenture','Wells Fargo','Lehigh'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Finance'],
                    'hire_date': [2004, 2008, 2012, 2014,2020]})

departments = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR','Janitors'],
                    'employees': [100,200,1,2] })

current_employer = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'firm': ['Yahoo','Yahoo','Yahoo','Yahoo'],})

some_employees_alt = pd.DataFrame({'employee': ['Jonathan', 'Mary', 'Lisa', 'Sue'],
                    'firm': ['?','Lehigh','Accenture','Wells Fargo'],
                    'group': ['Accounting', 'CEO', 'Engineering', 'HR'],
                    'hire_date': [2010, 2019, 2012, 2014]})

In [9]:
some_employees

,employee,firm,group,hire_date
0,Bob,Citi,Accounting,2004
1,Jake,GS,Engineering,2008
2,Lisa,Accenture,Engineering,2012
3,Sue,Wells Fargo,HR,2014
4,Austin,Lehigh,Finance,2020


In [10]:
departments

,group,employees
0,Accounting,100
1,Engineering,200
2,HR,1
3,Janitors,2


In [11]:
current_employer

,employee,firm
0,Bob,Yahoo
1,Jake,Yahoo
2,Lisa,Yahoo
3,Sue,Yahoo


In [12]:
some_employees_alt

,employee,firm,group,hire_date
0,Jonathan,?,Accounting,2010
1,Mary,Lehigh,CEO,2019
2,Lisa,Accenture,Engineering,2012
3,Sue,Wells Fargo,HR,2014


In [16]:
#merge some_employees and department 
some_employees.merge(departments,how='inner', on='group') #object.merge --> object is "left" implicitly 
pd.merge(left=some_employees,right=departments,how='left',on='group') #pd.merge(l,r) is equivalennt to df.merge(r) above


pd.merge(left=some_employees,right=departments,how='inner',on='group') #four - austin dead!
pd.merge(left=some_employees,right=departments,how='left',on='group') #whew, austin lives
pd.merge(left=some_employees,right=departments,how='right',on='group') #kills austin but gives janitors
pd.merge(left=some_employees,right=departments,how='outer',on='group') #gives both austin & janitors

### both RIGHT and OUTER produce observations without "employee" identifiers!
### this can't be, shouldn't be in an "employees" dataset 

,employee,firm,group,hire_date,employees
0,Bob,Citi,Accounting,2004.0,100.0
1,Jake,GS,Engineering,2008.0,200.0
2,Lisa,Accenture,Engineering,2012.0,200.0
3,Sue,Wells Fargo,HR,2014.0,1.0
4,Austin,Lehigh,Finance,2020.0,NaN
5,NaN,NaN,Janitors,NaN,2.0


In [25]:
#merge some_employees with current_employer
#merges when a variable is in both datasets, but is actually different 
pd.merge(left=some_employees,right=current_employer,how='inner',on='employee')


#two options: (1) suffix parameter (2) rename after the fact 

pd.merge(left=some_employees,right=current_employer,how='left',on='employee',
         suffixes = ('_prior','_current')) #suffix: one way to rename the columns

#renaming after the fact
a = pd.merge(left=some_employees,right=current_employer,how='inner',on='employee')
a = a.rename(columns = {'firm_x':'firm_prior',
         'firm_y': 'firm_current'}) #saying that firm_x will become firm_prior
a

,employee,firm_prior,group,hire_date,firm_current
0,Bob,Citi,Accounting,2004,Yahoo
1,Jake,GS,Engineering,2008,Yahoo
2,Lisa,Accenture,Engineering,2012,Yahoo
3,Sue,Wells Fargo,HR,2014,Yahoo


## Missing Data

what should we do? It depends!


In [26]:
import pandas as pd
import numpy as np

df = pd.DataFrame({"A":[12, 4, 5, None, 1], 
                   "B":[None, 2, 54, 3, None], 
                   "C":[20, 16, None, 3, 8], 
                   "D":[14, 3, None, None, 6]}) 

In [27]:
df

,A,B,C,D
0,12.0,NaN,20.0,14.0
1,4.0,2.0,16.0,3.0
2,5.0,54.0,NaN,NaN
3,NaN,3.0,3.0,NaN
4,1.0,NaN,8.0,6.0


In [35]:
df.fillna(0) #fills the whole dataset of blanks with 0


df2=df.copy()
df2['A'] = df['A'].fillna(0) #fills only the A column with 0, one variable at a time 
df2

df.fillna(df.mean()) #fill all blanks with mean

df.fillna(df.median()) #fill all with median of each column

df.fillna(method='ffill') #fill in from that of previous row in the column 



,A,B,C,D
0,12.0,NaN,20.0,14.0
1,4.0,2.0,16.0,3.0
2,5.0,54.0,16.0,3.0
3,5.0,3.0,3.0,3.0
4,1.0,3.0,8.0,6.0


In [37]:
#what if these are firms!

df4 = df.copy()
df4['firm']= 1
df5 = df.copy()
df5['firm'] = 2
df6= df4.append(df5).reset_index()


In [39]:
df6.fillna(method='ffill') #this is wrong, beacause row 5 for columm B should still be blank since its the start of firm 2

,index,A,B,C,D,firm
0,0,12.0,NaN,20.0,14.0,1
1,1,4.0,2.0,16.0,3.0,1
2,2,5.0,54.0,16.0,3.0,1
3,3,5.0,3.0,3.0,3.0,1
4,4,1.0,3.0,8.0,6.0,1
5,0,12.0,3.0,20.0,14.0,2
6,1,4.0,2.0,16.0,3.0,2
7,2,5.0,54.0,16.0,3.0,2
8,3,5.0,3.0,3.0,3.0,2
9,4,1.0,3.0,8.0,6.0,2


In [45]:
#how to fix it???

df6.groupby('firm').fillna(method='ffill')

,index,A,B,C,D
0,0,12.0,NaN,20.0,14.0
1,1,4.0,2.0,16.0,3.0
2,2,5.0,54.0,NaN,NaN
3,3,NaN,3.0,3.0,NaN
4,4,1.0,NaN,8.0,6.0
5,0,12.0,NaN,20.0,14.0
6,1,4.0,2.0,16.0,3.0
7,2,5.0,54.0,NaN,NaN
8,3,NaN,3.0,3.0,NaN
9,4,1.0,NaN,8.0,6.0
